In [2]:
import numpy as np

from keras.models import Sequential

from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D

from keras.preprocessing.image import ImageDataGenerator

from keras.utils import np_utils

from keras.datasets import cifar10

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 642s 4us/step


In [7]:
print(X_train.shape)
print(y_train.shape)

(50000, 32, 32, 3)
(50000, 1)


In [10]:
# 图片的长宽，以及分类的数量（十分类）
height, width, nb_class = 32, 32, 10
# 对图片的RGB通道做归一化
X_train = X_train / 255
X_test = X_test / 255.
# 将标签y转为分类
y_train = np_utils.to_categorical(y_train, nb_class)
y_test = np_utils.to_categorical(y_test, nb_class)
#  图像数据生成器，这里会根据已有图片，进行平移，旋转等操作生成一批新的图片用来训练，这个可以扩充数据集以及减少过拟合
gen = ImageDataGenerator()

In [17]:
#搭建网络
model = Sequential()
#  the dimensionality of the output space is 32(该层卷积输出尺寸)
# 卷积核的size为3，激活函数为relu，relu图像为max(0, x)
model.add(Conv2D(32, 3, padding='same', input_shape=X_train.shape[1:], activation='relu'))
# 第二层卷积
model.add(Conv2D(32, 3, activation='relu'))
# MaxPool，池化处理，2x2中最大的值选中
model.add(MaxPool2D(2))
# Dropout，增加一个Dropout层。每次训练时，都随机失活一定的神经元，这样可以防止过拟合 。一般会过滤掉50%的数据，这里过滤掉25%
model.add(Dropout(0.25))
#再加入两个卷积，一个池化和一个失活
model.add(Conv2D(64, 3, padding='same', activation='relu'))
model.add(Conv2D(64, 3, activation='relu'))
model.add(MaxPool2D(2))
model.add(Dropout(0.25))
# 然后添加展开层，它可以将这个tensor展开成一个一维张量，就是向量
model.add(Flatten())
# Dense就是全连接层，开始多层感知器，这一层有512个神经元
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
# 输出层，这一层只有nb_class个神经元了
model.add(Dense(nb_class, activation='softmax'))

# 看看我们神经网络的结构
# 因为核是3，所以图片尺寸变成了32 - 2 * ((3-1) / 2) = 30
# 池化图片尺寸减半
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 13, 13, 64)        36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 6, 6, 64)          0         
__________

In [ ]:
# 开始训练
# 损失函数为categorical_crossentropy, 多分类常用的损失函数。计算输出张量和目标张量的类别交叉熵
# 交叉熵计算公式为H((1,0,0),(0.5,0.4,0.1)) = -(1*log0.5 + 0 * log0.4 + 0*log0.1 ) = 0.3
# metrics: List of metrics to be evaluated by the model, during training and testing. Typically you will use `metrics=['accuracy']`.
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
# 训练方法
batch_size = 32
model.fit_generator(gen.flow(X_train, y_train, batch_size=batch_size),
                   steps_per_epoch=X_train.shape[0],
                   epochs=20, verbose=1, validation_data=(X_test, y_test))

Epoch 1/20
 2220/50000 [>.............................] - ETA: 2:59:55 - loss: 1.4197 - acc: 0.4850

In [ ]:
# 模型预测，由于本地根本训练不了这么大的数据集，所以这里就仅仅把代码记录下来
y_preds = model.predict(X_test[:10])

cifar10_cats = {0: 'airplane', 1: 'automobile', 2: 'bird', 3: 'cat', 4: 'deer', 5: 'dog',
                6: 'frog', 7: 'horse', 8: 'ship', 9: 'truck'}

for i in range(10):
    print('正确结果: {}\n预测结果: {}\n'.format(cifar10_cats[np.argmax(y_test[i])], 
                                            cifar10_cats[np.argmax(y_preds[i])]))